In [68]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import scipy.stats as st
from scipy import stats 
from scipy.stats import linregress

In [69]:
# Load Data to dict
# Output File (CSV)
output_data_file = "output_data/household_income.csv"



In [70]:
# Save config information
url = "https://datausa.io/api/data?Geography=16000US4260000:tracts&measure=Household%20Income%20by%20Race,Household%20Income%20by%20Race%20Moe&Race=0"
# units = "imperial"


# Build query URL
household = f"{url}"
household


'https://datausa.io/api/data?Geography=16000US4260000:tracts&measure=Household%20Income%20by%20Race,Household%20Income%20by%20Race%20Moe&Race=0'

In [71]:
response = requests.get(household).json()
response

{'data': [{'ID Year': 2018,
   'Year': '2018',
   'ID Race': 0,
   'Race': 'Total',
   'Household Income by Race': 102448,
   'Household Income by Race Moe': 16740,
   'Geography': 'Census Tract 1, Philadelphia County, PA',
   'ID Geography': '14000US42101000100'},
  {'ID Year': 2018,
   'Year': '2018',
   'ID Race': 0,
   'Race': 'Total',
   'Household Income by Race': 47243,
   'Household Income by Race Moe': 14802.999999999998,
   'Geography': 'Census Tract 2, Philadelphia County, PA',
   'ID Geography': '14000US42101000200'},
  {'ID Year': 2018,
   'Year': '2018',
   'ID Race': 0,
   'Race': 'Total',
   'Household Income by Race': 91354,
   'Household Income by Race Moe': 18636,
   'Geography': 'Census Tract 3, Philadelphia County, PA',
   'ID Geography': '14000US42101000300'},
  {'ID Year': 2018,
   'Year': '2018',
   'ID Race': 0,
   'Race': 'Total',
   'Household Income by Race': 62821,
   'Household Income by Race Moe': 10734,
   'Geography': 'Census Tract 4.01, Philadelphia Co

In [72]:
household_data = response['data']
household_data

[{'ID Year': 2018,
  'Year': '2018',
  'ID Race': 0,
  'Race': 'Total',
  'Household Income by Race': 102448,
  'Household Income by Race Moe': 16740,
  'Geography': 'Census Tract 1, Philadelphia County, PA',
  'ID Geography': '14000US42101000100'},
 {'ID Year': 2018,
  'Year': '2018',
  'ID Race': 0,
  'Race': 'Total',
  'Household Income by Race': 47243,
  'Household Income by Race Moe': 14802.999999999998,
  'Geography': 'Census Tract 2, Philadelphia County, PA',
  'ID Geography': '14000US42101000200'},
 {'ID Year': 2018,
  'Year': '2018',
  'ID Race': 0,
  'Race': 'Total',
  'Household Income by Race': 91354,
  'Household Income by Race Moe': 18636,
  'Geography': 'Census Tract 3, Philadelphia County, PA',
  'ID Geography': '14000US42101000300'},
 {'ID Year': 2018,
  'Year': '2018',
  'ID Race': 0,
  'Race': 'Total',
  'Household Income by Race': 62821,
  'Household Income by Race Moe': 10734,
  'Geography': 'Census Tract 4.01, Philadelphia County, PA',
  'ID Geography': '14000US42

In [73]:
# Set empty lists to hold data needed 

ID_Geography = []
Household_Income_by_Race = []
ID_Year = []


# Loop through each salary info
for info in household_data:
    ID_Geography.append(info['ID Geography'])
    Household_Income_by_Race.append(info['Household Income by Race'])
    ID_Year.append(info['ID Year'])
   



In [74]:
#create table to put see data
household_summary = {"ID_Geography":ID_Geography, "Household_Income_by_Race": Household_Income_by_Race, "ID_Year": ID_Year}
household_summary_df = pd.DataFrame(household_summary)
household_summary_df.ID_Year

0       2018
1       2018
2       2018
3       2018
4       2018
        ... 
2243    2013
2244    2013
2245    2013
2246    2013
2247    2013
Name: ID_Year, Length: 2248, dtype: int64

In [75]:
household_summary_df.to_csv("household_income_summary.csv", encoding="utf-8", index=False)

In [83]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///philly.sqlite', echo=True)
sqlite_connection = engine.connect()

# Add primary key to "merged" table that contains all cleaned data
sqlite_connection.execute('DROP TABLE "household"' )
sqlite_connection.execute('CREATE TABLE "household" ( "index" INTEGER PRIMARY KEY, "ID_Geography" TEXT, "Household_Income_by_Race" INTEGER, "ID_Year" INTEGER )' )


2021-04-27 21:56:13,445 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-04-27 21:56:13,446 INFO sqlalchemy.engine.base.Engine ()
2021-04-27 21:56:13,447 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-04-27 21:56:13,448 INFO sqlalchemy.engine.base.Engine ()
2021-04-27 21:56:13,448 INFO sqlalchemy.engine.base.Engine DROP TABLE "household"
2021-04-27 21:56:13,449 INFO sqlalchemy.engine.base.Engine ()
2021-04-27 21:56:13,453 INFO sqlalchemy.engine.base.Engine COMMIT
2021-04-27 21:56:13,454 INFO sqlalchemy.engine.base.Engine CREATE TABLE "household" ( "index" INTEGER PRIMARY KEY, "ID_Geography" TEXT, "Household_Income_by_Race" INTEGER, "ID_Year" INTEGER )
2021-04-27 21:56:13,454 INFO sqlalchemy.engine.base.Engine ()
2021-04-27 21:56:13,457 INFO sqlalchemy.engine.base.Engine COMMIT


In [85]:
sqlite_table = "household"
household_summary_df.to_sql(sqlite_table, sqlite_connection, if_exists='append', index=True, )

2021-04-27 21:56:57,848 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("household")
2021-04-27 21:56:57,849 INFO sqlalchemy.engine.base.Engine ()
2021-04-27 21:56:57,852 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-04-27 21:56:57,861 INFO sqlalchemy.engine.base.Engine INSERT INTO household ("index", "ID_Geography", "Household_Income_by_Race", "ID_Year") VALUES (?, ?, ?, ?)
2021-04-27 21:56:57,862 INFO sqlalchemy.engine.base.Engine ((0, '14000US42101000100', 102448, 2018), (1, '14000US42101000200', 47243, 2018), (2, '14000US42101000300', 91354, 2018), (3, '14000US42101000401', 62821, 2018), (4, '14000US42101000402', 78648, 2018), (5, '14000US42101000500', 43702, 2018), (6, '14000US42101000600', 82833, 2018), (7, '14000US42101000700', 55250, 2018)  ... displaying 10 of 2248 total bound parameter sets ...  (2246, '14000US42101980100', 66250, 2013), (2247, '14000US42101980200', 100313, 2013))
2021-04-27 21:56:57,870 INFO sqlalchemy.engine.base.Engine COMMIT


In [86]:
sqlite_connection.close()